In [57]:
!pip -q install recbole ray kmeans_pytorch

In [14]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle

import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec, Caser
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
from recbole.quick_start import run_recbole

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:

users_df = pd.read_csv("/content/drive/MyDrive/interactions_processed_kion.csv")
items_df = pd.read_csv("/content/drive/MyDrive/items_processed_kion.csv")
interactions_df = pd.read_csv("/content/drive/MyDrive/interactions_processed_kion.csv")

In [6]:
interactions_df['t_dat'] = pd.to_datetime(interactions_df['last_watch_dt'], format="%Y-%m-%d")
interactions_df['timestamp'] = interactions_df.t_dat.values.astype(np.int64) // 10 ** 9

In [7]:
df = interactions_df[['user_id', 'item_id', 'timestamp']].rename(
    columns={'user_id': 'user_id:token', 'item_id': 'item_id:token', 'timestamp': 'timestamp:float'})

In [8]:
!mkdir recbox_data

In [9]:
df.to_csv('recbox_data/recbox_data.inter', index=False, sep='\t')

In [20]:
parameter_dict = {
    'data_path': '',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'device': 'GPU',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'neg_sampling': None,
    'epochs': 10,
    'eval_args': {
        'split': {'RS': [9, 0, 1]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}
}
config = Config(model='MultiVAE', dataset='recbox_data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
# logger.info(config)

command line args [-f /root/.local/share/jupyter/runtime/kernel-4ee6d0af-7037-4200-8ee9-d73632a60040.json] will not be used in RecBole


In [16]:
dataset = create_dataset(config)
logger.info(dataset)

In [17]:
# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

In [21]:
import time
from recbole.quick_start import run_recbole

model_list = ['MultiVAE', 'MultiDAE', 'MacridVAE',"BPR", "NeuMF","RecVAE", 'RepeatNet', "LightGCN"]  # Added other models

for model_name in model_list:
    print(f"Running {model_name}...")
    start_time = time.time()

    # Run RecBole with the specified model
    result = run_recbole(model=model_name, dataset="recbox_data", config_dict=parameter_dict)

    elapsed_time = time.time() - start_time
    print(f"{model_name} took {elapsed_time / 60:.2f} mins")
    print(result)


Running MultiVAE...


command line args [-f /root/.local/share/jupyter/runtime/kernel-4ee6d0af-7037-4200-8ee9-d73632a60040.json] will not be used in RecBole
command line args [-f /root/.local/share/jupyter/runtime/kernel-4ee6d0af-7037-4200-8ee9-d73632a60040.json] will not be used in RecBole
Max value of user's history interaction records has reached 20.9471766848816% of the total.
Max value of user's history interaction records has reached 20.9471766848816% of the total.
Evaluate   : 100%|███████████████████████████████████████████| 13354/13354 [00:49<00:00, 268.82it/s]


MultiVAE took 3.16 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.084), ('mrr@10', 0.1695), ('ndcg@10', 0.0825), ('hit@10', 0.3503), ('precision@10', 0.0467)])}
Running MultiDAE...


command line args [-f /root/.local/share/jupyter/runtime/kernel-4ee6d0af-7037-4200-8ee9-d73632a60040.json] will not be used in RecBole
command line args [-f /root/.local/share/jupyter/runtime/kernel-4ee6d0af-7037-4200-8ee9-d73632a60040.json] will not be used in RecBole
Max value of user's history interaction records has reached 20.9471766848816% of the total.
Max value of user's history interaction records has reached 20.9471766848816% of the total.
Evaluate   : 100%|███████████████████████████████████████████| 13354/13354 [00:47<00:00, 281.16it/s]


MultiDAE took 3.10 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0829), ('mrr@10', 0.1655), ('ndcg@10', 0.081), ('hit@10', 0.3438), ('precision@10', 0.0459)])}
Running MacridVAE...


...

NeuMF took 11.38 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.069), ('mrr@10', 0.1173), ('ndcg@10', 0.0605), ('hit@10', 0.3009), ('precision@10', 0.0381)])}
Running RecVAE...


command line args [-f /root/.local/share/jupyter/runtime/kernel-4ee6d0af-7037-4200-8ee9-d73632a60040.json] will not be used in RecBole
command line args [-f /root/.local/share/jupyter/runtime/kernel-4ee6d0af-7037-4200-8ee9-d73632a60040.json] will not be used in RecBole
Max value of user's history interaction records has reached 20.9471766848816% of the total.
Max value of user's history interaction records has reached 20.9471766848816% of the total.
Evaluate   : 100%|███████████████████████████████████████████| 13354/13354 [01:02<00:00, 215.37it/s]


RecVAE took 9.20 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0849), ('mrr@10', 0.1697), ('ndcg@10', 0.0828), ('hit@10', 0.3532), ('precision@10', 0.047)])}
Running RepeatNet...


command line args [-f /root/.local/share/jupyter/runtime/kernel-4ee6d0af-7037-4200-8ee9-d73632a60040.json] will not be used in RecBole
command line args [-f /root/.local/share/jupyter/runtime/kernel-4ee6d0af-7037-4200-8ee9-d73632a60040.json] will not be used in RecBole
Train     2:  65%|███████████████████████████████▏                | 483/743 [40:56<22:02,  5.09s/it]


KeyboardInterrupt: ignored

# Kyboard stoped cause it was taking too much time

Choosed MultiVAE

In [22]:
result = run_recbole(model='MultiVAE', dataset = 'recbox_data',config_dict = parameter_dict )

command line args [-f /root/.local/share/jupyter/runtime/kernel-4ee6d0af-7037-4200-8ee9-d73632a60040.json] will not be used in RecBole
command line args [-f /root/.local/share/jupyter/runtime/kernel-4ee6d0af-7037-4200-8ee9-d73632a60040.json] will not be used in RecBole
Max value of user's history interaction records has reached 20.9471766848816% of the total.
Max value of user's history interaction records has reached 20.9471766848816% of the total.
Evaluate   : 100%|███████████████████████████████████████████| 13354/13354 [01:00<00:00, 219.45it/s]


In [33]:
result

{'best_valid_score': -inf,
 'valid_score_bigger': True,
 'best_valid_result': None,
 'test_result': OrderedDict([('recall@10', 0.084),
              ('mrr@10', 0.1695),
              ('ndcg@10', 0.0825),
              ('hit@10', 0.3503),
              ('precision@10', 0.0467)])}

In [35]:
# After trained model 'model' and configuration 'config'
# Save the model state dictionary and other necessary information
import torch
from recbole.model.general_recommender.multivae import MultiVAE
model = MultiVAE(config, dataset)
torch.save({
    'state_dict': model.state_dict(),
    'config': config
}, 'recbole.pth')

Max value of user's history interaction records has reached 23.254401942926535% of the total.
Max value of user's history interaction records has reached 23.254401942926535% of the total.


In [41]:
# Load model and optimizer state dictionaries
checkpoint = torch.load('recbole.pth')
model.load_state_dict(checkpoint['state_dict'])


<All keys matched successfully>

In [52]:
import numpy as np

def recommend_items_to_user(external_user_id, dataset, model):
    if external_user_id in dataset.field2token_id[dataset.uid_field] and external_user_id != "[PAD]":
        # Map external user ID to internal user index
        internal_user_index = dataset.field2token_id[dataset.uid_field][external_user_id]

        # Extract user metadata features and interaction vector
        user_metadata_features = dataset.users_ohe_df.drop(["user_id"], axis=1).iloc[internal_user_index]
        user_interaction_vector = dataset.interactions_vec[internal_user_index]

        # Predict user vector using the trained user-to-vector model
        user_vector = dataset.u2v.predict(
            [np.array(user_metadata_features).reshape(1, -1), np.array(user_interaction_vector).reshape(1, -1)],
            verbose=False,
        )

        # Instead of calculating distance for all items, just select a random subset
        sampled_item_indices = np.random.choice(dataset.item_embeddings.shape[0], size=100, replace=False)
        sampled_item_embeddings = dataset.item_embeddings[sampled_item_indices, :]

        # Calculate distances between the user vector and sampled item embeddings
        distances = np.linalg.norm(user_vector - sampled_item_embeddings, axis=1)

        # Get the indices of the top 10 items from the sampled set
        topn_item_indices_sampled = np.argsort(distances)[:10]

        # Map internal item indices to item_ids
        topn_item_ids = [dataset.iid_to_item_id[iid] for iid in topn_item_indices_sampled]

        return topn_item_ids

    return []


In [57]:
recos = {}
users = dataset.field2token_id[dataset.uid_field]
for user_id in users:
    recommended_items = recommend_items_to_user(user_id, dataset, model)
    if recommended_items:
        recos[user_id] = recommended_items